In [60]:
import numpy as np
from matplotlib import pyplot as plt
from NeuralNetwork import NeuralNetwork
from utils import get_data, onehot_encoding, grid_search, save_dict_to_file, load_dict_from_file
from Layer import Layer, Input
from functions import accuracy, MSE, MEE
import pandas as pd

In [72]:
best_comb_filename = '/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/best_comb_cup10.pkl'
param_grid_filename = '/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/param_grid_cup4.pkl'

In [73]:
names = ['id', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 
         'feature_7', 'feature_8', 'feature_9', 'feature_10', 'target_x', 'target_y','target_z']

df = pd.read_csv('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/ML-CUP23-TR.csv', names=names, comment='#')

targets = ['target_x', 'target_y', 'target_z']
features = list(set(names) - {'id', 'target_x', 'target_y', 'target_z'})

df = df.sample(frac=1)
prova = df[:750]
X_train, y_train = prova[features].to_numpy().T, prova[targets].to_numpy().T
X_test, y_test = df[750:][features].to_numpy().T, df[750:][targets].to_numpy().T

In [74]:
best_comb = load_dict_from_file(best_comb_filename)
param_grid = load_dict_from_file(param_grid_filename)

In [75]:
# Print the best {how_many} hyperparameter sets by their {sorting_key}

how_many = 5 # how many elements to print
sorting_key = 'val_MEE_mean' # value on which the sorting is performed
p_best = []

p_to_sort = [[x, x['results'][sorting_key]] for x in param_grid]
p_best_sorted = sorted(p_to_sort, key=lambda x:x[1])

for p_comb,loss in p_best_sorted[:how_many]:
    p_best.append(p_comb)
    tmp_dict = p_comb.copy()
    results = tmp_dict.pop('results')
    print(f"val_loss = {results['val_loss_mean']:.2e} +- {results['val_loss_std']:.2e}")
    print(f"val_MEE = {results['val_MEE_mean']:.2e} +- {results['val_MEE_std']:.2e}")
    print(f'hyperparameters : {tmp_dict}')
    print(f'results : {results}')
    print('')

val_loss = 3.78e-01 +- 1.13e-01
val_MEE = 7.66e-01 +- 9.70e-02
hyperparameters : {'eta': 0.003, 'lam': 3e-05, 'alpha': 0.9, 'epochs': 500, 'n_batch': 150, 'scale_eta_batchsize': None, 'dim_hidden': 70, 'hidden_act_func': 'tanh', 'dim_hidden2': 70, 'hidden_act_func2': 'tanh', 'use_opt': 0, 'loss': 'MSE', 'nest': True, 'output_act_func': 'lin', 'elapsed_time': 815.6306340694427}
results : {'train_loss_mean': 0.09085061645304614, 'train_loss_std': 0.0034567101783981318, 'val_loss_mean': 0.37835700197048105, 'val_loss_std': 0.11343619843388787, 'train_MEE_mean': 0.43806295167109743, 'train_MEE_std': 0.011709663899263105, 'val_MEE_mean': 0.765610541097898, 'val_MEE_std': 0.09701997822883787}

val_loss = 4.46e-01 +- 8.27e-02
val_MEE = 8.49e-01 +- 4.68e-02
hyperparameters : {'eta': 0.003, 'lam': 3e-05, 'alpha': 0.7, 'epochs': 500, 'n_batch': 150, 'scale_eta_batchsize': None, 'dim_hidden': 70, 'hidden_act_func': 'tanh', 'dim_hidden2': 70, 'hidden_act_func2': 'tanh', 'use_opt': 0, 'loss': 'MSE'

In [55]:
# Permorm retraining on the best {how_many} sets and print loss and MEE graphs.

retraining_epochs = 250
print_best = True # print the best results on all graphs

for i,p_comb in enumerate(p_best):

    best_comb = p_comb.copy()

    best_comb['epochs'] = retraining_epochs

    results = best_comb.pop('results')
    if best_comb['n_batch'] == 'batch':
        best_comb['n_batch'] = X_train.shape[1]
    elapsed_time = best_comb.pop('elapsed_time')
    if best_comb['scale_eta_batchsize'] == 'lin':
        best_comb['eta'] = best_comb['eta'] * best_comb['n_batch']
    if best_comb['scale_eta_batchsize'] == 'sqrt':
        best_comb['eta'] = best_comb['eta'] * np.sqrt(best_comb['n_batch'])
    best_comb.pop('scale_eta_batchsize')

    # ----------------------- MODEL ----------------------------
    
    input_layer = Input(X_train.shape[0])
    hidden_layer = Layer(input_layer, best_comb.pop('dim_hidden'), best_comb.pop('hidden_act_func'))
    o = 2
    while True:
        if f'dim_hidden{o}' in best_comb.keys():
            hidden_layer = Layer(hidden_layer, best_comb.pop(f'dim_hidden{o}'), best_comb.pop(f'hidden_act_func{o}'))
            o += 1
        else: break
    output_layer = Layer(hidden_layer, 3, best_comb.pop('output_act_func'))

    model = NeuralNetwork(input_layer, output_layer, best_comb.pop('loss'), metrics=[MEE])
    history = model.retrain(X_train, y_train.reshape((3,X_train.shape[1])), test_data = [X_test,y_test.reshape((3,X_test.shape[1]))], **best_comb)

    # ----------------------------------------------------------

    if i == 0:
        best_train_loss, best_test_loss = history['train_loss'], history['test_loss']
        best_train_MEE, best_test_MEE = history['train_MEE'], history['test_MEE']

    plt.figure(i, figsize=(30,10))
    
    plt.subplot(1,2,1)
    plt.plot(history['train_loss'],label='train_loss')
    plt.plot(history['test_loss'], label='test_loss')
    if (i > 0) and (print_best == True):
        plt.plot(best_train_loss,'--',label='best_train_loss')
        plt.plot(best_test_loss, '--', label='best_test_loss')
    plt.yscale('log')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('Loss functions', size=15)
    plt.title('train_loss vs test_loss', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)

    plt.subplot(1,2,2)
    plt.plot(history['train_MEE'],label='train_MEE')
    plt.plot(history['test_MEE'], label='test_MEE')
    if (i > 0) and (print_best == True):
        plt.plot(best_train_MEE,'--',label='best_train_MEE')
        plt.plot(best_test_MEE, '--', label='best_test_MEE')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('MEE', size=15)
    plt.title('train_MEE vs test_MEE', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)

    plt.show()

Epoch 0: train_loss: 1.995e+31, test_loss: 1.489e+31; train_MEE: 2.591e+15, test_MEE: 2.176e+15  lr : 0.009


c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in leaky_relu (vectorized)
  outputs = ufunc(*inputs)


KeyboardInterrupt: 

In [ ]:
def function(seed):
    np.random.seed(seed)
    return np.random.uniform(0,10,10)

In [ ]:
print(function(232323))

[5.58526844 2.1846418  6.79919952 7.11004665 7.14118482 0.5398932
 0.13095949 8.0790181  8.17264435 3.85478113]


: 